In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
import matplotlib.pyplot as plt
from numpy.random import default_rng

In [2]:
class ivModel:
    def __init__(self, r, repo, ttm, S, K, option, option_price):
        self.repo = repo
        self.ttm = ttm  
        self.r = r
        self.S = S
        self.K = K
        self.option = option
        self.sigmahat = np.sqrt(2*abs(((np.log(self.S/self.K))+ (self.r - self.repo) * self.ttm) / self.ttm))
        self.option_price = option_price
        
    def fCalculation(self,sigma):
        
        d1 = (np.log(self.S/self.K) + (self.r - self.repo) * self.ttm) / (sigma*np.sqrt(self.ttm)) + 0.5 * sigma * np.sqrt(self.ttm)
        d2 = (np.log(self.S/self.K) + (self.r - self.repo) * self.ttm) / (sigma*np.sqrt(self.ttm)) - 0.5 * sigma * np.sqrt(self.ttm)
        Csigma = self.S * np.exp(-self.repo * self.ttm) * norm.cdf(d1) - self.K * np.exp(-self.r * self.ttm) * norm.cdf(d2)
        Psigma = self.K * np.exp(-self.r * self.ttm) * norm.cdf(-d2) - self.S * np.exp(-self.repo * self.ttm) * norm.cdf(-d1)
        if self.option == "Call":
            return Csigma - self.option_price
        if self.option == "Put":
            return Psigma - self.option_price
        
    def fdevCalculation(self,sigma):
        
        d1 = (np.log(self.S/self.K) + (self.r - self.repo) * self.ttm) / (sigma*np.sqrt(self.ttm)) + 0.5 * sigma * np.sqrt(self.ttm)
        dN = np.exp(-0.5* d1 * d1) /(np.sqrt(2*np.pi)) 
        
        return self.S * np.exp(-self.repo*self.ttm) * np.sqrt(self.ttm) * dN
    
        
    def ivCalculation(self,n):
        
        tol = 1e-10
        nmax = n
        nCurrent = 0
        fdivfdev = 1
        sigma = self.sigmahat
        try:
            for nCurrent in range(nmax):
                fdivfdev = self.fCalculation(sigma) / self.fdevCalculation(sigma)
                sigma = sigma - fdivfdev
                if sigma <0:
                    sigma = "NaN"
                    break
                if abs(fdivfdev) < tol:
                    break  
                nCurrent = nCurrent+1        
            return sigma
        except:
            print("Error")
            return "NaN"
    